<a href="https://colab.research.google.com/github/Adubi/BUS-DRIVER-ASSISTANCE-SYSTEM/blob/main/AUTONOMOUS_PUBLIC_BUS_DRIVER_ASSISTANCE_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OBJECT DETECTION

In [ ]:
# Load video from Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install gdown -U --no-cache-dir

In [ ]:
import gdown
!gdown 13Xvx-fZJb55vvT9tGaAa6ijE6fHgYIkl

In [ ]:
!unzip data.zip

In [ ]:
#get the video
!gdown 1JXsAxKjfTeBNvYRCV9ew1mAMccNklH4s

In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

# Create folder in colab
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    # copying images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
train_test_split('/content/data/') # change to your dataset folder

In [ ]:
!git clone https://github.com/kanyakornju/speed-prediction.git
%cd /content/speed-prediction/object-detection
!pip install -r requirements.txt

In [ ]:
!python setup.py

In [ ]:
!python train.py --epochs 2 --data dataset.yaml --cfg yolov5s.yaml

In [ ]:
!python detect.py --save-txt --weights runs/train/exp2/weights/best.pt --conf 0.6 --source '/content/speed-prediction/object-detection/VID-20230217-WA0000.mp4'

## OBJ DETECTION MODEL EVAUATION


In [ ]:
#  Check this directory for results of experiments i.e recall curve, confusion matrix e.t.c /content/speed-prediction/object-detection/runs/train/exp


# DISTANCE ESTIMATION

In [ ]:
%cd /content/speed-prediction/distance-estimation
!pip install -r requirements.txt

GENERATE TRAIN-TEST DATASET FOR DISTANCE ESTIMATION FROM annotations.csv TO BE USED FOR TRAINING ON KITTI DATASET MODEL

In [ ]:
'''
Generate train-test dataset for distance estimation
'''
import pandas as pd
from tqdm import tqdm
import os
import numpy as np

df = pd.read_csv('annotations.csv')
df = df.dropna()
new_df = df.loc[df['class'] != 'DontCare']
result_df = pd.DataFrame(columns=['filename', 'class', 'xmin', 'ymin', 'xmax', 'ymax', \
                           'angle', 'xloc', 'yloc', 'zloc'])

pbar = tqdm(total=new_df.shape[0], position=1)

for idx, row in new_df.iterrows():
    pbar.update(1)
    result_df.at[idx, 'filename'] = row['filename']
    result_df.at[idx, 'class'] = row['class']

    result_df.at[idx, 'xmin'] = int(row['xmin'])
    result_df.at[idx, 'ymin'] = int(row['ymin'])
    result_df.at[idx, 'xmax'] = int(row['xmax'])
    result_df.at[idx, 'ymax'] = int(row['ymax'])

    result_df.at[idx, 'angle'] = row['observation angle']
    result_df.at[idx, 'xloc'] = int(row['xloc'])
    result_df.at[idx, 'yloc'] = int(row['yloc'])
    result_df.at[idx, 'zloc'] = int(row['zloc'])

mask = np.random.rand(len(result_df)) < 0.9
train = result_df[mask]
test = result_df[~mask]

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
"Train the distance estimation model"
!python train.py --results models/ --train train.csv --test test.csv

CHANGE model(.json) AND weights(.h5) TO YOUR MODEL NAME FROM "distance-estimation/models" FOLDER

In [ ]:
'''if you want to continue to train the model'''
#!python training_continuer.py --model /content/vehicle-distance-estimation/distance-estimator/models/model@1681254624.json --weights /content/vehicle-distance-estimation/distance-estimator/models/model@1681254624.h5

In [ ]:

!python plot_history.py --filename models/model@1682174488_results.csv

In [ ]:
!python inference.py --data /content/speed-prediction/object-detection/results/data/data.csv --model models/model@1682174488.json --weights models/model@1682174488.h5 --results /content/speed-prediction/distance-estimation

TO VISUALIZE THE DISTANCE IN THE VIDEO


In [ ]:
!python visualizer.py --data /content/speed-prediction/distance-estimation/predictions.csv --frames /content/speed-prediction/object-detection/results/frames -fps 30 --results /content/speed-prediction/distance-estimation

IMPORTANT NOTE: If the video is too big it cannot visualize, you can use the code below to zip your image and use python code to create the video. If you just want to see the result in your image go to '/content/vehicle-distance-estimation/object-detector/results/frames' folder in your colab

In [ ]:
#!zip -r /content/frames.zip /content/speed-prediction/object-detection/results/frames

## SPEED ESTIMATION

In [ ]:
%cd /content/speed-prediction/speed-estimation

In [ ]:
!gdown 1ZqjwCGR3as00ut_EZkyO5U0kZ5WcSLaW
!unzip data.zip

In [ ]:
!python opticalflow.py --test /content/VID-20230217-WA0000.mp4

In [ ]:
!python train.py

In [ ]:
!python test.py --test /content/VID-20230217-WA0000.mp4

In [ ]:
import csv
import pandas as pd

# Open the input file and read the contents
with open('/content/speed-prediction/speed-estimation/data/test.txt', 'r') as file:
    contents = file.read().splitlines()

# Create a list of tuples where the first element is the frame number and the second is the speed value
data = [(i+1, float(speed)) for i, speed in enumerate(contents)]

# Open the output file and create a CSV writer object
with open('speed.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['frame', 'Speed'])

    # Write each row of data to the file
    for row in data:
        writer.writerow(row)

# merged two dataframe
df1 = pd.read_csv('/content/speed-prediction/distance-estimation/predictions.csv') # replace with your own dataset filename
df2 = pd.read_csv('speed.csv') # replace with your own dataset filename

merged_df = pd.merge(df1, df2, on='frame')
merged_df.to_csv('/content/speed_test.csv', index=False)

## SPEED PREDICTION

In [ ]:
import math
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('/content/speed_test.csv')
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['label'] = df['label'].replace(['car','zebra crossing','human','bus stop'], [0,1,2,3])
df.head()

In [ ]:
df['label'] = df['label'].map({0:'car', 1:'zebra crossing', 2:'human', 3:'bus stop'})
df = pd.get_dummies(df, columns = ['label'], prefix=' ',prefix_sep=' ')
df.insert (13, "bus stop", 0)
df = df.drop('frame', axis =1)
df.head()

In [ ]:
train_df = df.sample(frac=0.8, random_state = 0)
test_df = df.drop(train_df.index)

In [ ]:
train_df.shape

In [ ]:
train_df.shape

In [ ]:
train_features = train_df.copy()
test_features = test_df.copy()
x_train, y_train = train_df.drop('Speed', axis=1), train_df['Speed']
x_test, y_test = test_df.drop('Speed', axis=1), test_df['Speed']

In [ ]:
def scale_datasets(x_train, x_test):

  """
  Standard Scale test and train data
  Z - Score normalization
  """
  minmax_scaler = MinMaxScaler()
  x_train_scaled = pd.DataFrame(
      minmax_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      minmax_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled
x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)


# Creating model using the Sequential in tensorflow
def build_model_using_sequential():
  model = Sequential([
    Dense(64, input_shape = [13], kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(32, kernel_initializer='normal',  kernel_regularizer='l2', activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(16, kernel_initializer='normal', kernel_regularizer='l2', activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(8, kernel_initializer='normal', kernel_regularizer='l2', activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(1, kernel_initializer='normal', activation='linear')
  ])
  return model


# build the model
model = build_model_using_sequential()

In [ ]:
# loss function
msle = MeanSquaredLogarithmicError()

model.compile(
    loss=msle,
    optimizer=Adam(learning_rate=0.0005),
    metrics=[msle]
)

# Hyperparameter tuning ranges
epochs_range = range(100, 1001, 100)
batch_sizes_range = [32, 64, 128, 256, 512]

# Perform hyperparameter tuning
for epochs in epochs_range:
    for batch_size in batch_sizes_range:
        print(f"Training model with epochs={epochs}, batch_size={batch_size}")

        # Train the model with the current hyperparameters
        history = model.fit(
            x_train_scaled.values,
            y_train.values,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            verbose=0
        )

        # Print the loss and metrics for the last epoch
        print(f"Final Loss: {history.history['loss'][-1]}")
        print(f"Final Mean Squared Logarithmic Error: {history.history['mean_squared_logarithmic_error'][-1]}")
        print()

In [ ]:
# train the model
history = model.fit(
    x_train_scaled.values,
    y_train.values,
    epochs=1000,
    batch_size=256,
    validation_split=0.2
)

In [ ]:
msle_score = model.evaluate(x_train_scaled, y_train, verbose=0)
print("MSLE score:", msle_score)

In [ ]:
def plot_history(history, key):
  plt.plot(history.history[key])
  plt.plot(history.history['val_'+key])
  plt.xlabel("Epochs")
  plt.ylabel(key)
  plt.legend([key, 'val_'+key])
  plt.show()
# Plot the history
plot_history(history, 'mean_squared_logarithmic_error')

In [ ]:
x_test['prediction'] = model.predict(x_test_scaled)
x_test

In [ ]:
predictions = x_test['prediction']
test = test_features['Speed']
a = plt.axes(aspect='equal')
plt.scatter(test, predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [1, 15]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)